In [ ]:
#!/usr/bin/python
import glob
import time
import sys
import numpy as np
import scipy.misc
import argparse
import os
import warnings

In [ ]:
# Make sure that caffe is on the python path:
caffe_root = '/home/axj232/code/caffe-nv/'
sys.path.insert(0, caffe_root + 'python')

import caffe


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# configure plotting
plt.rcParams['figure.figsize'] = (20, 20)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
#setup some input paramters and provide defaults
parser = argparse.ArgumentParser(description='make output of files')
parser.add_argument('pattern',
                    help="Pattern to upload, for example, *.tif, or the name of a text file containing filenames to upload (use -i option)")

parser.add_argument('-p', '--patchsize', help="patchsize, default 32", default=32, type=int)
parser.add_argument('-o', '--outdir', help="outputdir, default ./output/", default="./output/", type=str)
parser.add_argument('-b', '--binary', help="binary mean file", default="DB_train.binaryproto", type=str)
parser.add_argument('-m', '--model', help="model", default="full_convolutional_net.caffemodel", type=str)
parser.add_argument('-y', '--deploy', help="ORIGINAL deploy file", default="deploy_full.prototxt", type=str)
parser.add_argument('-i', '--gpuid', help="id of gpu to use", default=0, type=int)
parser.add_argument('-v', '--batch', help="batch size", default=8192, type=int)
parser.add_argument('-c', '--cmask', help="use compuation mask", default=False, action="store_true")

#args = parser.parse_args()
args = parser.parse_args(["-v32768","-c","-p32","-bDB_train_1.binaryproto","-msnapshot_iter_71730.caffemodel","-ydeploy.prototxt","-o./out/","small.png"])

hwsize=args.patchsize/2
if not os.path.exists(args.outdir):
    os.makedirs(args.outdir)

In [ ]:
#load the binary file and conver it to the proper format
a = caffe.io.caffe_pb2.BlobProto()
file = open(args.binary, 'rb')
data = file.read()
a.ParseFromString(data)
means = a.data
means = np.asarray(means)
means = means.reshape(3, 32, 32)

#specify that we want to use the gpu
caffe.set_mode_gpu()
caffe.set_device(2)


#create the classifier
net = caffe.Classifier(args.deploy, args.model,
                       mean=means,
                       channel_swap=(2, 1, 0),
                       raw_scale=255,
                       image_dims=(args.patchsize, args.patchsize))




In [ ]:
files = sorted(glob.glob(args.pattern))
fname=files[0]
warnings.warn("only computing first file, for loop not implemented")

In [ ]:
newfname_prob = "%s/%s_prob.png" % (args.outdir,os.path.basename(fname)[:-4])
#see if it already exists, if it does skip it...this allows us to run multiple instanes at once
#if (os.path.exists(newfname_prob)):
#    continue
#print "working on file: \t %s" % fname

#save a flag file, so we can know that there is a process working on this particular file 

outputimage = np.zeros(shape=(10, 10))

scipy.misc.imsave(newfname_prob, outputimage)

image = caffe.io.load_image(fname)

print image.shape
if (args.cmask):
    mask = caffe.io.load_image("%s_mask.png"%fname[0:-4], color=False)
else:
    mask = np.ones(shape=(image.shape[0], image.shape[1], 1))

image = np.lib.pad(image, ((hwsize, hwsize), (hwsize, hwsize), (0, 0)), 'symmetric')
mask = np.lib.pad(mask, ((hwsize, hwsize), (hwsize, hwsize), (0, 0)), 'constant',
                  constant_values=((0, 0), (0, 0), (0, 0)))

outputimage_probs = np.zeros(shape=(image.shape[0],image.shape[1],3))

non_zeros = mask.nonzero()
num_non_zero = np.count_nonzero(mask)
print "this many nonzeros to compute [x] in [y] :\t%d\t%d\n" % (num_non_zero, mask.shape[0] * mask.shape[1])
rcs = []
patches = []
listi = 0
start_time = time.time()
start_time_iter = 0
print outputimage_probs.shape
while listi < num_non_zero:
    print outputimage_probs.shape
    print "(G) %s\t (%.3f,%.3f)\t %d of %d" % (
        fname, time.time() - start_time, time.time() - start_time_iter, listi, num_non_zero)
    start_time_iter = time.time()
    while len(patches) < args.batch:
        if (listi >= num_non_zero):
            break

        rowi = non_zeros[0][listi]
        coli = non_zeros[1][listi]
        patches.append(image[rowi - hwsize:rowi + hwsize, coli - hwsize:coli + hwsize, :])
        rcs.append([rowi, coli])
        listi += 1
    print "(R) %s\t (%.3f,%.3f)\t %d of %d" % (
        fname, time.time() - start_time, time.time() - start_time_iter, listi, num_non_zero)
    prediction = net.predict(patches,oversample=False)
    print "(D) %s\t (%.3f,%.3f)\t %d of %d" % (
        fname, time.time() - start_time, time.time() - start_time_iter, listi, num_non_zero)
    rcs = np.array(rcs)
    outputimage_probs[rcs[:, 0], rcs[:, 1], 0:2] = np.squeeze(prediction)
    patches = []
    rcs = []



outputimage_probs = outputimage_probs[hwsize:-hwsize, hwsize:-hwsize, :]

scipy.misc.imsave(newfname_prob, outputimage_probs)


#(D) small.png	 (410.838,410.837)	 11663 of 11663 #yes overfeat
#(D) small.png	 (56.741,56.740)	 11663 of 11663 #no overfeat